In [72]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, GRU, Bidirectional, Conv1D, MaxPooling1D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from keras import regularizers
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [3]:
data = pd.read_csv('/Users/inigoparra/Desktop/carpeta sin título/clean-master.csv')

In [74]:
train_data, test_data = train_test_split(data, test_size=0.1, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.1, random_state=42)

X_train = train_data['text']
y_train = train_data['label']
X_test = test_data['text']
y_test = test_data['label']
X_val = val_data['text']
y_val = val_data['label']

In [37]:
with open('/Users/inigoparra/Desktop/GitHub Repositories/WiBaSets/stopwords.txt', 'r') as f:
    stopwords = set(f.read().splitlines())

def preprocess_text(text):
    if not isinstance(text, str):
        text = str(text)
        
    tokens = word_tokenize(text)
    filtered_tokens = [token for token in tokens if token.lower() not in stopwords]
    return ' '.join(filtered_tokens)

X_train = X_train.apply(preprocess_text)
X_test = X_test.apply(preprocess_text)
X_val = X_val.apply(preprocess_text)

print(X_train[0])

lana nabarmentzen bakea demokrazia euskal gizartearen ongizatea lortzeko


In [77]:
import re
import nltk
from nltk.corpus import stopwords

def clean_text(text):

    text = str(text).lower()
    text = re.sub(r'<.*?>', '', text)  
    text = re.sub(r'[0-9]', '', text)  
    text = re.sub(r'[^\w\s]', '', text)

    stop_words = set(stopwords.words('basque'))
    
    tokens = nltk.word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]

    text = ' '.join(tokens)
    return text

X_train = X_train.apply(clean_text)
X_test = X_test.apply(clean_text)
X_val = X_val.apply(clean_text)

print(X_train[6])

ondarrutar arrazoi bozkatzeko


In [78]:
max_features = 20500
max_len = 100
embedding_dim = 300

tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)
X_val_seq = tokenizer.texts_to_sequences(X_val)

X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)
X_val_pad = pad_sequences(X_val_seq, maxlen=max_len)

In [83]:
model = Sequential()
model.add(Embedding(max_features, embedding_dim, input_length=max_len))
model.add(Bidirectional(LSTM(32, return_sequences=True,
               kernel_regularizer=regularizers.l2(0.02), 
               recurrent_regularizer=regularizers.l2(0.02))))
model.add(Dropout(0.6))
model.add(GRU(16))
model.add(Dropout(0.6))
model.add(Dense(3, activation='softmax'))

optimizer = Adam(lr=0.00001)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [84]:
batch_size = 32
epochs = 100
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(
    X_train_pad, y_train,
    validation_data=(X_val_pad, y_val),
    batch_size=batch_size,
    epochs=epochs,
    callbacks=[early_stopping]
)

Epoch 1/100
215/215 [==============================] - 19s 76ms/step - loss: 9.3239 - accuracy: 0.3312 - val_loss: 9.1109 - val_accuracy: 0.3508
Epoch 2/100
215/215 [==============================] - 16s 73ms/step - loss: 8.9069 - accuracy: 0.3531 - val_loss: 8.7048 - val_accuracy: 0.3599
Epoch 3/100
215/215 [==============================] - 16s 73ms/step - loss: 8.5106 - accuracy: 0.3646 - val_loss: 8.3180 - val_accuracy: 0.3613
Epoch 4/100
215/215 [==============================] - 16s 72ms/step - loss: 8.1325 - accuracy: 0.3709 - val_loss: 7.9493 - val_accuracy: 0.3534
Epoch 5/100
215/215 [==============================] - 16s 72ms/step - loss: 7.7719 - accuracy: 0.3777 - val_loss: 7.5973 - val_accuracy: 0.3534
Epoch 6/100
215/215 [==============================] - 16s 73ms/step - loss: 7.4269 - accuracy: 0.3834 - val_loss: 7.2611 - val_accuracy: 0.3521
Epoch 7/100
215/215 [==============================] - 16s 73ms/step - loss: 7.0983 - accuracy: 0.3790 - val_loss: 6.9398 - val_ac

In [85]:
y_predict = model.predict(X_val_pad)
y_predict_classes = np.argmax(y_predict, axis=1)
print(classification_report(y_val, y_predict_classes, digits=3))

24/24 [==============================] - 1s 16ms/step
              precision    recall  f1-score   support

           0      0.000     0.000     0.000       211
           1      0.363     0.398     0.380       266
           2      0.377     0.620     0.469       287

    accuracy                          0.372       764
   macro avg      0.247     0.340     0.283       764
weighted avg      0.268     0.372     0.308       764



/Users/inigoparra/opt/anaconda3/envs/TF/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/inigoparra/opt/anaconda3/envs/TF/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/inigoparra/opt/anaconda3/envs/TF/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

In [86]:
loss, accuracy = model.evaluate(X_test_pad, y_test)
print(f'Test loss: {loss}, Test accuracy: {accuracy}')

27/27 [==============================] - 0s 15ms/step - loss: 1.1799 - accuracy: 0.3887
Test loss: 1.1798779964447021, Test accuracy: 0.38869258761405945
